!pip install pycaret[full]

In [9]:
import pandas as pd
# dataset = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/C조/사과/hybrid_apple_FV.xlsx', usecols = 'C:L'))
# target = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/C조/사과/hybrid_apple_FV.xlsx', usecols = 'M'))
dataset = pd.read_excel('./hybrid_apple_FV.xlsx', usecols = 'C:M')
data = dataset.loc[:, ['mean_temp', 'feb', 'mar', 'apr', 'latitude', 'elevation', 'bud_burst_j', 'full_blm_j']]

In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mean_temp    339 non-null    float64
 1   feb          339 non-null    float64
 2   mar          339 non-null    float64
 3   apr          339 non-null    float64
 4   precp        339 non-null    float64
 5   latitude     339 non-null    float64
 6   elevation    339 non-null    int64  
 7   northness_c  339 non-null    float64
 8   slope        339 non-null    float64
 9   bud_burst_j  339 non-null    int64  
 10  full_blm_j   339 non-null    int64  
dtypes: float64(8), int64(3)
memory usage: 29.3 KB


In [11]:
# from pycaret.classification import *
# !pip install markupsafe==2.0.1
from pycaret.regression import *
exp = setup(data = dataset, 
            target = 'full_blm_j',
            train_size = 0.75,
            session_id = 123,
            )

,Description,Value
0,Session id,123
1,Target,full_blm_j
2,Target type,Regression
3,Data shape,"(339, 11)"
4,Train data shape,"(254, 11)"
5,Test data shape,"(85, 11)"
6,Numeric features,10
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [12]:
best = compare_models()
best

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,1.6600,4.9417,2.1175,0.8989,0.0183,0.0143,0.5010
et,Extra Trees Regressor,1.6426,4.9849,2.1387,0.8966,0.0186,0.0143,0.0390
gbr,Gradient Boosting Regressor,1.7157,5.6161,2.2533,0.8812,0.0195,0.0148,0.0270
lightgbm,Light Gradient Boosting Machine,1.9195,6.3461,2.4156,0.8670,0.0208,0.0166,0.0170
rf,Random Forest Regressor,1.9006,6.7270,2.4881,0.8555,0.0215,0.0164,0.0450
ada,AdaBoost Regressor,2.0781,7.2414,2.6278,0.8469,0.0228,0.0180,0.0260
xgboost,Extreme Gradient Boosting,1.9489,7.2660,2.5945,0.8404,0.0223,0.0168,0.1890
br,Bayesian Ridge,2.1813,8.3956,2.7761,0.8249,0.0235,0.0187,0.0160
ridge,Ridge Regression,2.1851,8.3978,2.7814,0.8238,0.0235,0.0187,0.0130
lr,Linear Regression,2.1857,8.4004,2.7822,0.8237,0.0235,0.0187,0.0170


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [17]:
et = create_model('et', cross_validation = False)
gbr = create_model('gbr', cross_validation = False)
catboost = create_model('catboost', cross_validation = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,2.2781,8.7372,2.9559,0.7784,0.0256,0.0196


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,2.5478,10.5246,3.2442,0.7331,0.0278,0.0218


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,2.3065,9.2912,3.0481,0.7644,0.0262,0.0198


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
tuned_et = tune_model(et, optimize = 'R2', n_iter = 10) # n_iter : train 데이터셋 반복 횟수
tuned_gbr = tune_model(gbr, optimize = 'R2', n_iter = 10)
tuned_catboost = tune_model(catboost, optimize = 'R2', n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.8301,15.2607,3.9065,0.7129,0.0328,0.0240
1,1.7518,7.0391,2.6531,0.8120,0.0237,0.0157
2,2.0919,5.8380,2.4162,0.8956,0.0207,0.0181
3,1.2790,2.3544,1.5344,0.9182,0.0132,0.0111
4,2.1956,9.5065,3.0833,0.7550,0.0264,0.0189
5,1.9638,5.9326,2.4357,0.8999,0.0206,0.0170
6,1.6904,4.6116,2.1475,0.8511,0.0187,0.0146
7,3.3465,17.6342,4.1993,0.7667,0.0352,0.0282
8,1.3052,2.9628,1.7213,0.9303,0.0151,0.0116


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.6756,14.4639,3.8031,0.7279,0.0314,0.0222
1,1.1248,1.8381,1.3558,0.9509,0.0119,0.0098
2,1.3867,2.9716,1.7238,0.9469,0.0151,0.0121
3,1.2525,2.4555,1.5670,0.9147,0.0136,0.0109
4,1.7333,5.0888,2.2558,0.8688,0.0198,0.0151
5,1.5433,3.3753,1.8372,0.9431,0.0159,0.0135
6,1.7401,5.8903,2.4270,0.8098,0.0210,0.0151
7,2.0869,7.6040,2.7575,0.8994,0.0241,0.0180
8,1.3981,4.4249,2.1035,0.8958,0.0184,0.0123


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.6131,13.1535,3.6268,0.7525,0.0306,0.0221
1,1.4799,3.3349,1.8262,0.9109,0.0159,0.0129
2,1.8535,4.9210,2.2183,0.9120,0.0196,0.0162
3,1.3069,2.6854,1.6387,0.9067,0.0139,0.0113
4,1.7433,5.0592,2.2493,0.8696,0.0193,0.0150
5,1.7795,4.4959,2.1204,0.9242,0.0185,0.0156
6,1.6112,5.0075,2.2377,0.8383,0.0193,0.0139
7,2.5509,10.4638,3.2348,0.8616,0.0280,0.0219
8,1.3170,3.1212,1.7667,0.9265,0.0152,0.0115


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [19]:
## 모델 블렌딩
blender = blend_models(estimator_list = [tuned_et, tuned_gbr, tuned_catboost], optimize = 'R2')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.5860,13.9623,3.7366,0.7373,0.0309,0.0216
1,1.0740,1.6218,1.2735,0.9567,0.0112,0.0094
2,1.3877,2.7566,1.6603,0.9507,0.0142,0.0120
3,1.1766,1.8482,1.3595,0.9358,0.0117,0.0102
4,1.5046,4.1939,2.0479,0.8919,0.0181,0.0131
5,1.4823,3.1597,1.7775,0.9467,0.0154,0.0129
6,1.4786,4.4383,2.1067,0.8567,0.0183,0.0128
7,2.1663,7.1630,2.6764,0.9053,0.0233,0.0187
8,1.2539,3.0126,1.7357,0.9291,0.0151,0.0110


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
final_model = finalize_model(blender)
pred = predict_model(final_model, data = dataset)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.8662,1.3031,1.1415,0.9727,0.0099,0.0075
